In [447]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
import pprint as pp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pprint as pp
%matplotlib inline

In [448]:
file = 'Liquor_Authority_Quarterly_List_of_Active_Licenses.csv'

df = pd.read_csv(file)
df.shape #total records

(51235, 21)

In [449]:
#Examine datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51235 entries, 0 to 51234
Data columns (total 21 columns):
License Serial Number                        51235 non-null int64
License Type Name                            51235 non-null object
License Class Code                           51189 non-null float64
License Type Code                            51235 non-null object
Agency Zone Office Name                      51235 non-null object
Agency Zone Office Number                    51235 non-null int64
County Name (Licensee)                       51235 non-null object
Premises Name                                51235 non-null object
Doing Business As (DBA)                      35795 non-null object
Actual Address of Premises (Address1)        51235 non-null object
Additional Address Information (Address2)    11154 non-null object
City                                         51235 non-null object
State                                        51235 non-null object
Zip                  

In [450]:
#Convert Dates to Datetime & Zip Codes to Ints
df['License Effective Date'] = pd.to_datetime(df['License Effective Date'])
df['License Original Issue Date'] = pd.to_datetime(df['License Original Issue Date']) 
df['License Expiration Date'] = pd.to_datetime(df['License Expiration Date'])
df['Zip'] = df.Zip.astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51235 entries, 0 to 51234
Data columns (total 21 columns):
License Serial Number                        51235 non-null int64
License Type Name                            51235 non-null object
License Class Code                           51189 non-null float64
License Type Code                            51235 non-null object
Agency Zone Office Name                      51235 non-null object
Agency Zone Office Number                    51235 non-null int64
County Name (Licensee)                       51235 non-null object
Premises Name                                51235 non-null object
Doing Business As (DBA)                      35795 non-null object
Actual Address of Premises (Address1)        51235 non-null object
Additional Address Information (Address2)    11154 non-null object
City                                         51235 non-null object
State                                        51235 non-null object
Zip                  

In [264]:
#TYPES OF LICENSES (NY STATE)
df['License Type Name'].value_counts().head()

ON-PREMISES LIQUOR          16234
GROCERY BEER, WINE PROD      9569
RESTAURANT WINE              5230
LIQUOR STORE                 3276
GROCERY CHAIN, WINE PROD     2435
Name: License Type Name, dtype: int64

In [452]:
file1 = 'nyc_zcta10_to_puma10.xls'
df1 = pd.read_excel(file1, 'zctas_assigned')
new_dict = df1.set_index('zcta10').T.to_dict('list')

zips = {}

for key, value in new_dict.items():
    zips.update({key: value[4]})

pp.pprint(zips)

{10001: 'Chelsea, Clinton & Midtown Business District',
 10002: 'Chinatown & Lower East Side',
 10003: 'Chinatown & Lower East Side',
 10004: 'Battery Park City, Greenwich Village & Soho',
 10005: 'Battery Park City, Greenwich Village & Soho',
 10006: 'Battery Park City, Greenwich Village & Soho',
 10007: 'Battery Park City, Greenwich Village & Soho',
 10009: 'Chinatown & Lower East Side',
 10010: 'Murray Hill, Gramercy & Stuyvesant Town',
 10011: 'Chelsea, Clinton & Midtown Business District',
 10012: 'Battery Park City, Greenwich Village & Soho',
 10013: 'Battery Park City, Greenwich Village & Soho',
 10014: 'Battery Park City, Greenwich Village & Soho',
 10016: 'Murray Hill, Gramercy & Stuyvesant Town',
 10017: 'Murray Hill, Gramercy & Stuyvesant Town',
 10018: 'Chelsea, Clinton & Midtown Business District',
 10019: 'Chelsea, Clinton & Midtown Business District',
 10020: 'Chelsea, Clinton & Midtown Business District',
 10021: 'Upper East Side',
 10022: 'Murray Hill, Gramercy & Stuyv

In [453]:
list_of_zips = [str(key) for key, value in zips.items()]

In [454]:
nyc = df.loc[(df['Zip'].isin(list_of_zips))]

In [455]:
nyc['License Type Name'].value_counts().head(10)

ON-PREMISES LIQUOR         6904
GROCERY BEER, WINE PROD    5395
RESTAURANT WINE            2983
LIQUOR STORE               1444
GROCERY STORE BEER         1243
DRUG CHAIN, WINE PROD       537
TAVERN WINE                 406
EATING PLACE BEER           377
HOTEL LIQUOR                310
CATERING ESTABLISHMENT      214
Name: License Type Name, dtype: int64

In [456]:
list_of_counties = ['BRONX', 'KINGS', 'NEW YORK', 'QUEENS', 'RICHMOND']
df.loc[(df['County Name (Licensee)'].isin(list_of_counties))]['License Type Name'].value_counts().head(10)

ON-PREMISES LIQUOR         6991
GROCERY BEER, WINE PROD    5416
RESTAURANT WINE            3004
LIQUOR STORE               1448
GROCERY STORE BEER         1247
DRUG CHAIN, WINE PROD       548
TAVERN WINE                 418
EATING PLACE BEER           384
HOTEL LIQUOR                316
CATERING ESTABLISHMENT      237
Name: License Type Name, dtype: int64

In [457]:
zips_df = pd.DataFrame.from_dict(zips, orient='index')
zips_df = zips_df.reset_index()
zips_df.columns = ['Zip', 'Neighborhood']
zips_df.head()
zips_df.info()

,Zip,Neighborhood
0,10451,"Hunts Point, Longwood & Melrose"
1,10452,"Concourse, Highbridge & Mount Eden"
2,10453,"Morris Heights, Fordham South & Mount Hope"
3,10454,"Hunts Point, Longwood & Melrose"
4,10455,"Hunts Point, Longwood & Melrose"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 2 columns):
Zip             211 non-null int64
Neighborhood    211 non-null object
dtypes: int64(1), object(1)
memory usage: 3.4+ KB


In [458]:
nyc_on = nyc[nyc['License Type Name'] == 'ON-PREMISES LIQUOR']
main_df = nyc_on.groupby('Zip').count().sort_values('License Serial Number', ascending=False)['License Serial Number']
main_df = main_df.reset_index()
main_df.columns = ['Zip', 'Number of Licenses']
main_df['Zip'] = main_df.Zip.astype('int64')
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 2 columns):
Zip                   188 non-null int64
Number of Licenses    188 non-null int64
dtypes: int64(2)
memory usage: 3.0 KB


In [459]:
main = zips_df.merge(main_df)

In [460]:
final_table = main.sort_values('Number of Licenses', ascending=False)

In [461]:
final_table = final_table.set_index('Zip')

In [443]:
final_table.head(25)
final_table['Number of Licenses'].sum()

,Neighborhood,Number of Licenses
Zip,,
10019,"Chelsea, Clinton & Midtown Business District",273
10003,Chinatown & Lower East Side,237
10036,"Chelsea, Clinton & Midtown Business District",235
11211,Greenpoint & Williamsburg,227
10014,"Battery Park City, Greenwich Village & Soho",202
10013,"Battery Park City, Greenwich Village & Soho",180
10022,"Murray Hill, Gramercy & Stuyvesant Town",178
10011,"Chelsea, Clinton & Midtown Business District",175
10016,"Murray Hill, Gramercy & Stuyvesant Town",174


6904

In [462]:
grouped_by_neighborhood = final_table.groupby('Neighborhood').sum()

In [463]:
grouped_by_neighborhood = grouped_by_neighborhood.sort_values('Number of Licenses', ascending = False).head(10)

In [464]:
df1 = pd.read_excel(file1, 'zctas_assigned')
df_puma = pd.DataFrame([df1.puma10, df1.nameshort]).transpose()

In [465]:
grouped_by_neighborhood = grouped_by_neighborhood.reset_index()

In [467]:
df_puma.columns = ['Puma', 'Neighborhood']
df_puma = df_puma.drop_duplicates('Puma', keep='first')
df_puma.head()

,Puma,Neighborhood
0,3710,"Hunts Point, Longwood & Melrose"
1,3708,"Concourse, Highbridge & Mount Eden"
2,3707,"Morris Heights, Fordham South & Mount Hope"
5,3705,"Belmont, Crotona Park East & East Tremont"
7,3706,"Bedford Park, Fordham North & Norwood"


In [316]:
final['Number of Licenses'].sum()

6904

In [484]:
final.head(25)

,Neighborhood,Number of Licenses,Puma
0,"Chelsea, Clinton & Midtown Business District",985,3807
1,"Battery Park City, Greenwich Village & Soho",710,3810
2,"Murray Hill, Gramercy & Stuyvesant Town",540,3808
3,Chinatown & Lower East Side,516,3809
4,Greenpoint & Williamsburg,312,4001
5,Upper East Side,287,3805
6,Astoria & Long Island City,277,4101
7,Brooklyn Heights & Fort Greene,260,4004
8,Upper West Side & West Side,230,3806
9,"Park Slope, Carroll Gardens & Red Hook",174,4005


In [318]:
file_puma = 'demo_2016acs5yr_puma.xlsx'

puma = pd.read_excel(file_puma)
puma_df = pd.DataFrame(puma)
puma_df.head()

,GeoType,GeogName,GeoID,Borough,Pop_1E,Pop_1M,Pop_1C,Pop_1P,Pop_1Z,MaleE,...,AsnOAsnE,AsnOAsnM,AsnOAsnC,AsnOAsnP,AsnOAsnZ,Asn2plE,Asn2plM,Asn2plC,Asn2plP,Asn2plZ
0,PUMA2010,NYC-Brooklyn Community District 1--Greenpoint ...,4001,Brooklyn,157433,2644,1.0,100,NaN,78095,...,200,137.0,41.7,1.7,1.2,159,80.0,30.6,1.4,0.7
1,PUMA2010,NYC-Brooklyn Community District 4--Bushwick PU...,4002,Brooklyn,135837,3022,1.4,100,NaN,67768,...,17,19.0,67.9,0.3,0.3,143,111.0,47.2,2.1,1.6
2,PUMA2010,NYC-Brooklyn Community District 3--Bedford-Stu...,4003,Brooklyn,142689,2723,1.2,100,NaN,67613,...,56,42.0,45.6,1.2,0.9,88,62.0,42.8,1.9,1.3
3,PUMA2010,NYC-Brooklyn Community District 2--Brooklyn He...,4004,Brooklyn,135679,2054,0.9,100,NaN,64316,...,143,128.0,54.6,1.1,1.0,316,124.0,23.9,2.5,1.0
4,PUMA2010,"NYC-Brooklyn Community District 6--Park Slope,...",4005,Brooklyn,116344,1738,0.9,100,NaN,55601,...,71,53.0,45.2,0.9,0.7,139,100.0,43.7,1.8,1.3


In [469]:
puma_df_2 = pd.DataFrame([puma_df.GeoID, puma_df.Borough, puma_df.Pop_1E]).transpose()

In [470]:
puma_df_2.columns = ['Puma', 'Borough', 'Population']

In [471]:
final_final = final.merge(puma_df_2, on='Puma')

In [485]:
final_final['Licenses per 1k People'] = (final_final['Number of Licenses'] / (final_final['Population'] / 1000))

In [483]:
final_final.groupby('Borough').aggregate('sum', axis='columns')

,Neighborhood,Number of Licenses,Puma,Population,Licenses per 1k People
Borough,,,,,
Brooklyn,Greenpoint & WilliamsburgBrooklyn Heights & Fo...,1521,72171,2606852,10.793629
Manhattan,"Chelsea, Clinton & Midtown Business DistrictBa...",3529,38055,1634989,22.220123
Queens,"Astoria & Long Island CityFlushing, Murray Hil...",1219,57505,2310039,7.505988
Staten Island,"New Springville & South BeachTottenville, Grea...",266,11706,473324,1.725021
The Bronx,"Co-op City, Pelham Bay & SchuylervilleBedford ...",369,37055,1436785,2.716361


In [486]:
final_final.head(10)

,Neighborhood,Number of Licenses,Puma,Borough,Population,Licenses per 1k People
0,"Chelsea, Clinton & Midtown Business District",985,3807,Manhattan,149309,6.59706
1,"Battery Park City, Greenwich Village & Soho",710,3810,Manhattan,155488,4.56627
2,"Murray Hill, Gramercy & Stuyvesant Town",540,3808,Manhattan,146750,3.67973
3,Chinatown & Lower East Side,516,3809,Manhattan,160709,3.21077
4,Greenpoint & Williamsburg,312,4001,Brooklyn,157433,1.9818
5,Upper East Side,287,3805,Manhattan,221922,1.29325
6,Astoria & Long Island City,277,4101,Queens,172468,1.6061
7,Brooklyn Heights & Fort Greene,260,4004,Brooklyn,135679,1.91629
8,Upper West Side & West Side,230,3806,Manhattan,193715,1.18731
9,"Park Slope, Carroll Gardens & Red Hook",174,4005,Brooklyn,116344,1.49556


In [478]:
final_final.groupby('Borough')['Population'].sum()

Borough
Brooklyn         2606852
Manhattan        1634989
Queens           2310039
Staten Island     473324
The Bronx        1436785
Name: Population, dtype: int64

In [441]:
nyc_on.loc[nyc_on['Zip'] == '10019']


,License Serial Number,License Type Name,License Class Code,License Type Code,Agency Zone Office Name,Agency Zone Office Number,County Name (Licensee),Premises Name,Doing Business As (DBA),Actual Address of Premises (Address1),...,City,State,Zip,License Certificate Number,License Original Issue Date,License Effective Date,License Expiration Date,Latitude,Longitude,Location
2041,1026162,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,AGORA RESTAURANT CORP,MOLYVOS,871 SEVENTH AVE,...,NEW YORK,NY,10019,891865,1997-06-23,2018-06-01,2020-05-31,40.764558,-73.980628,"(40.76455781, -73.98062826)"
2042,1026166,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,APPLEJACK COFFEE SHOP INC,NaN,1725 BROADWAY,...,NEW YORK,NY,10019,902112,1981-04-01,2019-03-01,2021-02-28,40.764597,-73.982327,"(40.76459696, -73.98232709)"
2043,1026170,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,ARRIBA ARRIBA MEXICAN RESTAURANTS INC,ARRIBA ARRIBA MEXICAN REST,762 9TH AVENUE,...,NEW YORK,NY,10019,880566,1984-12-04,2017-12-01,2019-11-30,40.764100,-73.988390,"(40.76409998, -73.98839017)"
2044,1026178,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,BARRAJA INC,THALIA,250 W 50TH STREET,...,NEW YORK,NY,10019,882660,1999-09-08,2017-10-10,2019-08-31,40.761821,-73.985123,"(40.7618207, -73.98512304)"
2045,1026179,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,BENIHANA NATIONAL CORP,BENIHANA OF TOKYO WEST,47 W 56TH STREET,...,NEW YORK,NY,10019,875629,1995-08-09,2017-08-01,2019-07-31,40.762976,-73.975943,"(40.76297601, -73.97594298)"
2046,1026184,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,BRICCO CORP,NaN,304 WEST 56TH ST,...,NEW YORK,NY,10019,883125,NaT,2017-11-01,2019-10-31,NaN,NaN,NaN
2047,1026185,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,BROADWAY BLUES 50TH ST CORP,NaN,226 W 50TH STREET,...,NEW YORK,NY,10019,873068,1996-04-30,2017-04-01,2019-03-31,40.761623,-73.984652,"(40.76162254, -73.98465233)"
2048,1026187,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,BROOKLYN DINER USA L P,BROOKLYN DINER USA INC G P,212 WEST 57TH ST,...,NEW YORK,NY,10019,902539,1996-03-20,2019-03-01,2021-02-28,40.765798,-73.980821,"(40.76579825, -73.9808215)"
2049,1026193,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,CARNEGIE BAR AND BOOKS LTD,THE CARNEGIE CLUB,156 W 56TH ST,...,NEW YORK,NY,10019,883576,1997-02-25,2018-02-01,2020-01-31,40.764442,-73.979606,"(40.76444236, -73.9796055)"
2050,1026196,ON-PREMISES LIQUOR,252.0,OP,New York City,1,NEW YORK,CERTA GENTE LLC,NaN,325 W 51ST STREET,...,NEW YORK,NY,10019,875356,1999-06-09,2017-06-01,2019-05-31,40.763267,-73.986336,"(40.76326697, -73.98633634)"


In [326]:
nyc_clean = nyc_on.dropna(subset=['Latitude', 'Longitude'])
nyc_clean.shape

(6782, 21)

In [496]:
file_zip = 'us-zip-code-latitude-and-longitude.csv'

master_zip = pd.read_csv(file_zip, ';')
master_zip.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,49544,Grand Rapids,MI,42.999561,-85.75371,-5,1,"42.999561, -85.75371"
1,49648,Kewadin,MI,45.000254,-85.36510,-5,1,"45.000254, -85.3651"
2,49654,Leland,MI,45.023384,-85.75697,-5,1,"45.023384, -85.75697"
3,49682,Suttons Bay,MI,44.993085,-85.63635,-5,1,"44.993085, -85.63635"
4,49747,Hubbard Lake,MI,44.827421,-83.61368,-5,1,"44.827421, -83.61368"


In [512]:
lat_long = final_table.merge(master_zip, on='Zip', how='inner')
lat_long = lat_long.drop(columns=['Timezone', 'Daylight savings time flag'])
lat_long.head(100)

,Zip,Neighborhood,Number of Licenses,City,State,Latitude,Longitude,geopoint
0,10019,"Chelsea, Clinton & Midtown Business District",273,New York,NY,40.765714,-73.98560,"40.765714, -73.9856"
1,10003,Chinatown & Lower East Side,237,New York,NY,40.732509,-73.98935,"40.732509, -73.98935"
2,10036,"Chelsea, Clinton & Midtown Business District",235,New York,NY,40.759511,-73.99019,"40.759511, -73.99019"
3,11211,Greenpoint & Williamsburg,227,Brooklyn,NY,40.712090,-73.95427,"40.71209, -73.95427"
4,10014,"Battery Park City, Greenwich Village & Soho",202,New York,NY,40.734718,-74.00532,"40.734718, -74.00532"
5,10013,"Battery Park City, Greenwich Village & Soho",180,New York,NY,40.720666,-74.00526,"40.720666, -74.00526"
6,10022,"Murray Hill, Gramercy & Stuyvesant Town",178,New York,NY,40.759015,-73.96732,"40.759015, -73.96732"
7,10011,"Chelsea, Clinton & Midtown Business District",175,New York,NY,40.741012,-74.00012,"40.741012, -74.00012"
8,10016,"Murray Hill, Gramercy & Stuyvesant Town",174,New York,NY,40.746180,-73.97759,"40.74618, -73.97759"
9,10012,"Battery Park City, Greenwich Village & Soho",169,New York,NY,40.725960,-73.99834,"40.72596, -73.99834"


In [502]:
import folium
nyc_map = folium.Map(location=[40.7128, -74.006],
                     tiles = "Stamen Toner",
                     zoom_start = 11)

nyc_map

In [504]:
data = pd.DataFrame({
'lat':lat_long['Latitude'],
'lon':lat_long['Longitude'],
'radius': lat_long['Number of Licenses']
})

for i in range(0,len(data)):
    folium.Circle(
      location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
      radius=data.iloc[i]['radius'],
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(nyc_map)

In [505]:
nyc_map

In [516]:
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[40.7128, -74.006],
                     tiles = "Stamen Toner",
                    zoom_start = 11) 

# Ensure you're handing it floats
lat_long['Latitude'] = lat_long['Latitude'].astype(float)
lat_long['Longitude'] = lat_long['Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = lat_long[['Latitude', 'Longitude']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude'])

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data,radius=30).add_to(map_hooray)

# Display the map
map_hooray

In [513]:
nyc_clean

,License Serial Number,License Type Name,License Class Code,License Type Code,Agency Zone Office Name,Agency Zone Office Number,County Name (Licensee),Premises Name,Doing Business As (DBA),Actual Address of Premises (Address1),...,City,State,Zip,License Certificate Number,License Original Issue Date,License Effective Date,License Expiration Date,Latitude,Longitude,Location
200,1003823,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,KENNETH MCPARTLAN TAVERN CORP,GLEN ROY TAVERN,145 E 149TH STREET,...,BRONX,NY,10451,901767,1979-06-04,2019-03-01,2021-02-28,40.818850,-73.928023,"(40.81884963, -73.92802257)"
201,1003824,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,GLACKENS BAR & GRILL CORP,GLACKENS BAR & GRILL,135 E 149TH STREET,...,BRONX,NY,10451,886107,2005-06-22,2018-04-01,2020-03-31,40.818940,-73.928375,"(40.81893952, -73.9283752)"
202,1003854,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,STEBAR FOOD CORP,WALNUT BUSSTOP COFFEE SHOP,881 883 E 134TH STREET,...,BRONX,NY,10454,902017,NaT,2019-03-01,2021-02-28,40.800418,-73.910742,"(40.80041785, -73.9107424)"
203,1003874,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,LA LECHONERA CRIOLLA INC,NaN,1709 WEBSTER AVE,...,BRONX,NY,10457,876365,2011-12-07,2017-07-03,2019-05-31,40.843393,-73.903519,"(40.84339267, -73.90351916)"
204,1003876,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,EL DIAMANTE RESTAURANT CORP,EL DIAMANTE RESTAURANT,4371 3RD AVE,...,BRONX,NY,10457,877658,2013-01-17,2017-08-01,2019-07-31,40.850702,-73.894171,"(40.85070212, -73.8941709)"
205,1003885,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,DOMINICKS BAR & RESTAURANT INC,NaN,2335 ARTHUR AVENUE,...,BRONX,NY,10458,874887,NaT,2017-06-01,2019-05-31,40.854135,-73.888850,"(40.85413489, -73.88884964)"
206,1003897,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,MARIOS RESTAURANT INC,MARIOS RESTAURANT,2342 ARTHUR AVENUE,...,BRONX,NY,10458,883629,1980-09-17,2018-03-01,2020-02-29,40.854125,-73.888684,"(40.85412539, -73.88868431)"
207,1003900,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,MEXICAN ENTERTAINMENT OF WEBSTER INC,NaN,2649 WEBSTER AVENUE,...,BRONX,NY,10458,900691,1997-12-29,2019-01-09,2020-11-30,40.863580,-73.889058,"(40.8635801, -73.88905778)"
208,1003904,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,P A R T RESTAURANT CORP,PASQUALES RIGOLETTO,2311 ARTHUR AVE,...,BRONX,NY,10458,900637,NaT,2019-01-07,2020-12-31,40.853520,-73.889388,"(40.85351969, -73.88938787)"
209,1003909,ON-PREMISES LIQUOR,252.0,OP,New York City,1,BRONX,S&N ARTHUR AVENUE INC,NaN,2331 ARTHUR AVE,...,BRONX,NY,10458,880214,NaT,2017-12-01,2019-11-30,40.854004,-73.888877,"(40.85400383, -73.88887748)"


In [524]:
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[40.7128, -74.006],
                     tiles = "Stamen Toner",
                    zoom_start = 11) 

# Ensure you're handing it floats
nyc_clean['Latitude'] = nyc_clean['Latitude'].astype(float)
nyc_clean['Longitude'] = nyc_clean['Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = nyc_clean[['Latitude', 'Longitude']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude'])

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data, radius=20, blur=18).add_to(map_hooray)

# Display the map
map_hooray